In [1]:
# Import packages
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point
import matplotlib as mpl
import matplotlib.pyplot as plt 
import seaborn as sns
#import libpysal as ps
from statsmodels.tsa.seasonal import STL

In [159]:
#all_visits = pd.read_csv("toR.csv",parse_dates=['datetime'])#POI in whole BMA

In [2]:
all_visits = pd.read_csv("top3.csv",parse_dates=['datetime'])#POI in centres

In [184]:
all_visits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102403 entries, 0 to 102402
Data columns (total 15 columns):
 #   Column     Non-Null Count   Dtype                                 
---  ------     --------------   -----                                 
 0   X          102403 non-null  int64                                 
 1   poi        102403 non-null  int64                                 
 2   day        102403 non-null  int64                                 
 3   date       102403 non-null  object                                
 4   datetime   102403 non-null  datetime64[ns, pytz.FixedOffset(-300)]
 5   weekday    102403 non-null  int64                                 
 6   time       102403 non-null  object                                
 7   dist       102403 non-null  float64                               
 8   type       102403 non-null  object                                
 9   uid        102403 non-null  object                                
 10  from_ID    102403 no

In [5]:
all_visits.head()

,Unnamed: 0.1,Unnamed: 0,uid,POI_index,datetime,type,lat,lng,home_pings,home_tile_ID,work_pings,work_tile_ID,tot_pings,month,distance,weekday,day,stratum
0,64893,64893,077480D3-7C5E-46AE-8CAF-E949E78C5A35,4201,2020-01-01 00:00:00-05:00,bars,4.673600,-74.08080,344,114,100,112,10996,1,5899.702473,2,1,4.675979
1,64933,64933,077480D3-7C5E-46AE-8CAF-E949E78C5A35,2185,2020-01-01 00:00:00-05:00,dentists,4.673611,-74.08081,344,114,100,112,10996,1,5898.887773,2,1,4.675979
2,65522,65522,077480d3-7c5e-46ae-8caf-e949e78c5a35,1754,2020-01-01 00:00:00-05:00,conveniences,4.676800,-74.08130,174,114,79,112,6160,1,5643.532653,2,1,4.675979
3,65550,65550,077480d3-7c5e-46ae-8caf-e949e78c5a35,2537,2020-01-01 00:00:00-05:00,hgs,4.673000,-74.08000,174,114,79,112,6160,1,5992.393360,2,1,4.675979
4,521670,521670,3f5e8df1-da24-4259-a16c-d34cc31f9bb2,7772,2020-01-01 00:00:00-05:00,restaurants,4.853300,-74.05670,70,120,538,229,3081,1,12217.889000,2,1,3.631530


In [3]:
strat_stats = pd.read_csv("raw_data/strat_stats.csv")

In [29]:
strat_stats

,day,count,av,median,low_ct,med_ct,hi_ct
0,1,557118,2.824008,2.839070,230467,297584,29067
1,2,442660,2.928896,2.899845,173067,234201,35392
2,3,356399,3.023536,2.951414,126606,195036,34757
3,4,424599,2.940255,2.915527,163046,226168,35385
4,5,333147,3.028093,2.938682,115740,182310,35097
...,...,...,...,...,...,...,...
542,543,354710,2.749812,2.781147,146207,193295,15208
543,544,285929,2.694051,2.678130,130061,142935,12933
544,545,239117,2.787076,2.875949,94958,132651,11508
545,546,144807,2.724574,2.781147,61698,76955,6154


In [38]:
#all_visits is the dataset I already gave you
df0=pd.DataFrame({"poi":all_visits["POI_index"],"day":all_visits["datetime"].dt.dayofyear+365*(all_visits["datetime"].dt.year==2021),"datetime":all_visits["datetime"],"type":all_visits["type"],"uid":all_visits["uid"]})
df0

,poi,day,datetime,type,uid
0,4201,1,2020-01-01 00:00:00-05:00,bars,077480D3-7C5E-46AE-8CAF-E949E78C5A35
1,2185,1,2020-01-01 00:00:00-05:00,dentists,077480D3-7C5E-46AE-8CAF-E949E78C5A35
2,1754,1,2020-01-01 00:00:00-05:00,conveniences,077480d3-7c5e-46ae-8caf-e949e78c5a35
3,2537,1,2020-01-01 00:00:00-05:00,hgs,077480d3-7c5e-46ae-8caf-e949e78c5a35
4,7772,1,2020-01-01 00:00:00-05:00,restaurants,3f5e8df1-da24-4259-a16c-d34cc31f9bb2
...,...,...,...,...,...
1048570,471,546,2021-06-30 18:36:00-05:00,cloths,14fb39bc-e5a8-4638-97a9-7602d8a64229
1048571,1505,546,2021-06-30 18:36:00-05:00,restaurants,1f064ba0-201f-4c25-8105-aa2738c8f11b
1048572,4238,546,2021-06-30 18:48:00-05:00,cafes,0cd97ae5-df49-455f-9bda-9c7b5f36659e
1048573,471,546,2021-06-30 18:48:00-05:00,cloths,14fb39bc-e5a8-4638-97a9-7602d8a64229


In [12]:
df0 = all_visits.copy()
df0 = df0.loc[(df0['datetime'].dt.time > pd.to_datetime('2020-01-01 08:00:00').time())&
              (df0['datetime'].dt.time < pd.to_datetime('2020-01-01 20:00:00').time())]
df0

,X,poi,day,date,datetime,weekday,time,dist,type,uid,from_ID,from_stra,to_ID,to_stra,region
0,25,4162,1,01/01/2020,2020-01-01 08:48:00-05:00,2,08:48:00,12171.394210,supers,4EAEB9B0-B3E2-4233-9C2F-67388BF8FD75,956,3.239237,89,2.448219,2
1,26,5301,1,01/01/2020,2020-01-01 08:48:00-05:00,2,08:48:00,12174.830080,conveniences,4EAEB9B0-B3E2-4233-9C2F-67388BF8FD75,956,3.239237,89,2.448219,2
2,36,473,1,01/01/2020,2020-01-01 09:00:00-05:00,2,09:00:00,9.779007,dentists,3377246f-d423-4b07-b27a-d33de01482df,857,NaN,733,4.084951,1
3,43,15321,1,01/01/2020,2020-01-01 09:12:00-05:00,2,09:12:00,3818.154614,restaurants,3fc5a49e-402e-46d2-a068-f880bc868d30,365,2.724138,295,2.962969,3
4,46,3414,1,01/01/2020,2020-01-01 09:12:00-05:00,2,09:12:00,3484.126749,dentists,69F021B9-2F02-4B25-A839-3EF0CACF07BA,797,NaN,578,4.052930,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102398,804604,3711,366,31/12/2020,2020-12-31 19:00:00-05:00,3,19:00:00,6498.648171,hgs,51516954-af1f-4bba-adc9-225113d4af74,614,2.963171,292,3.002224,2
102399,804613,3654,366,31/12/2020,2020-12-31 19:12:00-05:00,3,19:12:00,221.498215,restaurants,60C9AF03-7AF9-4BE9-A47B-8FFCA621C7D8,139,4.981607,139,4.980514,2
102400,804627,7770,366,31/12/2020,2020-12-31 19:24:00-05:00,3,19:24:00,4486.252705,hgs,5899533d-2870-43e5-92aa-74dbb6ed16f8,602,1.883372,492,2.792792,2
102401,804653,1879,366,31/12/2020,2020-12-31 19:48:00-05:00,3,19:48:00,6488.663041,electrics,51516954-af1f-4bba-adc9-225113d4af74,614,2.963171,292,3.002224,2


In [37]:
#only count unique person x visits, examine restaurant visits only (you could choose another POI type) and take aggegrate counts by day
df1=df0.copy()
df1=df1.loc[(df1["type"]=="electrics")
            &(df1['region']==3)]#only needed when analyse the centres

df1=df1.drop_duplicates(subset=["day","poi","uid"])
df1=df1.groupby(["day"]).count()["uid"].reset_index()
df1=df1.sort_values(by="day")
df1.loc[df1['day']==171]

,day,uid
143,171,2


In [38]:
#compute the fraction of visits per day relative to the total number of pings in the entire sample area - call this level
df2=pd.merge(df1,strat_stats,on="day")
df2["lvl"]=df2["uid"]/df2["count"]
df2=df2.set_index("day")

In [39]:
df1a=df2.copy()
df1a.iloc[156:186] # 2 weeks before and after

q=np.array(df1a["lvl"])

In [40]:
#delete the observation of the tax free day - we will compute the expected volume via interpolation
q[170]=np.nan
df1a["lvl"]=q
df1a["lvl"]=df1a["lvl"].interpolate()

In [41]:
#STL interpolation
res = STL(df1a["lvl"], robust = True,period=7).fit() 

#compute observed volume as fraction of expected volume given the interpolation
print(df2.loc[df2.index==171]["lvl"].values[0])
print(res.trend.iloc[171])


1.75000175000175e-06
8.705057404116592e-07
